 # Mancala AI - Aloken Chaudhari

This project is based on the implementation of Mancala done within HW 7. It will fully develop a Mancala AI player with the use the aima repository.

In [1]:
import math
import time as tm
import sys
sys.path.append('./aima-python')

from games4e import *

#### **Game Implementation from HW 7**

In [2]:
import math
import random
from games4e import Game, GameState  # Assuming these are implemented in the aima repository

class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        """Initialize the Mancala board and game state."""
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        self.board[pits_per_player] = 0  # Player 1's Mancala
        self.board[-1] = 0  # Player 2's Mancala
        self.players = 2
        self.current_player = 1
        self.p1_pits_index = range(0, pits_per_player)
        self.p2_pits_index = range(pits_per_player + 1, len(self.board) - 1)

    def display_board(self):
        """Display the Mancala board."""
        print(f"P2: {self.board[-2:self.pits_per_player:-1]} | Mancala: {self.board[-1]}")
        print(f"P1: {self.board[0:self.pits_per_player]} | Mancala: {self.board[self.pits_per_player]}")
        print(f"Current Player: {'P1' if self.current_player == 1 else 'P2'}")

    def valid_moves(self):
        """Return a list of valid moves for the current player."""
        if self.current_player == 1:
            return [i for i in self.p1_pits_index if self.board[i] > 0]
        else:
            return [i for i in self.p2_pits_index if self.board[i] > 0]

    def play(self, pit):
        """Perform a move for the current player."""
        stones = self.board[pit]
        self.board[pit] = 0
        index = pit

        while stones > 0:
            index = (index + 1) % len(self.board)
            if (self.current_player == 1 and index == len(self.board) - 1) or \
               (self.current_player == 2 and index == self.pits_per_player):
                continue
            self.board[index] += 1
            stones -= 1

        # Capture logic
        if self.current_player == 1 and index in self.p1_pits_index and self.board[index] == 1:
            opposite_index = len(self.board) - 2 - index
            self.board[self.pits_per_player] += self.board[opposite_index] + 1
            self.board[opposite_index] = 0
        elif self.current_player == 2 and index in self.p2_pits_index and self.board[index] == 1:
            opposite_index = len(self.board) - 2 - index
            self.board[-1] += self.board[opposite_index] + 1
            self.board[opposite_index] = 0

        # End-of-turn checks
        if index != (self.pits_per_player if self.current_player == 1 else len(self.board) - 1):
            self.current_player = 2 if self.current_player == 1 else 1

    def is_game_over(self):
        """Check if the game is over."""
        return all(self.board[i] == 0 for i in self.p1_pits_index) or \
               all(self.board[i] == 0 for i in self.p2_pits_index)

    def winning_eval(self):
        """Evaluate the winning state of the game."""
        if self.is_game_over():
            p1_score = self.board[self.pits_per_player] + sum(self.board[i] for i in self.p1_pits_index)
            p2_score = self.board[-1] + sum(self.board[i] for i in self.p2_pits_index)
            return p1_score - p2_score
        return None

    def random_move_generator(self):
        """Generate a random valid move for the current player."""
        valid = self.valid_moves()
        if valid:
            return random.choice(valid)
        return None

def play_game(game, strategy1, strategy2):
    """Play a full game using two strategies."""
    while not game.is_game_over():
        game.display_board()
        if game.current_player == 1:
            move = strategy1(game)
        else:
            move = strategy2(game)

        if move is not None:
            game.play(move)
        else:
            print(f"Player {game.current_player} has no valid moves.")
            break

    game.display_board()
    winner = game.winning_eval()
    if winner > 0:
        print("Player 1 wins!")
    elif winner < 0:
        print("Player 2 wins!")
    else:
        print("It's a tie!")


#### **Random vs. Random**

This code will return the win percentage of 100 games with 2 players choosing random moves 

In [3]:
games = 100
p1_wins = 0  
p2_wins = 0 
total_moves = 0  

for _ in range(games):
    game = Mancala() 
    moves = 0  
    
    while not game.winning_eval(): 
        pit = game.random_move_generator()  
        
        if pit is None:
            print("No valid moves available. Ending the game.")
            break
        
        game.play(pit)  
        moves += 1 
    
    total_moves += moves  
    
    if game.current_player == 1:  
        p2_wins += 1  
    else:
        p1_wins += 1  

p1_win_percentage = (p1_wins / games) * 100
p2_win_percentage = (p2_wins / games) * 100
average_moves = total_moves / games

print(f"Player 1 (Random) won: {p1_win_percentage:.2f}% of the time")
print(f"Player 2 (Random) won: {p2_win_percentage:.2f}% of the time")
print(f"On average, it took {average_moves:.2f} moves to win.")

No valid moves available. Ending the game.
No valid moves available. Ending the game.
No valid moves available. Ending the game.
Player 1 (Random) won: 55.00% of the time
Player 2 (Random) won: 45.00% of the time
On average, it took 50.97 moves to win.


#### **Subclassing the Game Class from games4e.ipynb**

This is an implementation of the game class to fit our Mancala game implementation.

In [4]:
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

class MancalaGame(Game):
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        """A game is similar to a problem, but it has a utility for each
        state and a terminal test instead of a path cost and a goal
        test. To create a game, subclass this class and implement actions,
        result, utility, and terminal_test. You may override display and
        successors or you can inherit their default methods. You will also
        need to set the .initial attribute to the initial state; this can
        be done in the constructor."""
        self.pits_per_player = pits_per_player
        # self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        # self.board[pits_per_player] = 0  # Player 1's Mancala
        # self.board[-1] = 0  # Player 2's Mancala
        self.players = 2
        self.current_player = 1
        self.p1_pits_index = range(0, pits_per_player)
        self.p2_pits_index = range(pits_per_player + 1, 2 * pits_per_player + 1 - 1)
        
        temp_board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        temp_board[pits_per_player] = 0
        temp_board[2 * pits_per_player + 1] = 0
        
        self.initial = GameState(
            to_move=1,
            utility=0,
            board= temp_board,
            moves=[],
        )
    

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        if state.to_move == 1:
            pits = range(self.pits_per_player)
        else:
            pits = range(self.pits_per_player + 1, len(state.board) - 1)

        return [pit for pit in pits if state.board[pit] > 0]

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        board = state.board[:]
        player = state.to_move
        stones = board[move]
        board[move] = 0
        current_index = move

        while stones > 0:
            current_index = (current_index + 1) % len(board)
            # Skip the opponent's Mancala
            if (player == 1 and current_index == len(board) - 1) or \
               (player == 2 and current_index == self.pits_per_player):
                continue
            board[current_index] += 1
            stones -= 1

        # Capture logic
        if player == 1 and self.pits_per_player > current_index >= 0 and board[current_index] == 1:
            opposite_index = len(board) - 2 - current_index
            board[self.pits_per_player] += board[opposite_index] + 1
            board[current_index] = 0
            board[opposite_index] = 0
        elif player == 2 and len(board) - 2 > current_index > self.pits_per_player and board[current_index] == 1:
            opposite_index = len(board) - 2 - current_index
            board[-1] += board[opposite_index] + 1
            board[current_index] = 0
            board[opposite_index] = 0

        # Check end of game
        if self.terminal_test(state):
            for i in range(self.pits_per_player):
                board[self.pits_per_player] += board[i]
                board[-1] += board[-2 - i]
                board[i] = board[-2 - i] = 0

        # Switch player
        next_player = 2 if player == 1 else 1
        return GameState(to_move=next_player, utility=self.utility(state, player), board=board, moves=state.moves + [move])

    def utility(self, state, player):
        """Return the value of this final state to player."""
        p1_score = state.board[self.pits_per_player]
        p2_score = state.board[-1]
        if p1_score > p2_score:
            return 1 if player == 1 else -1
        elif p2_score > p1_score:
            return 1 if player == 2 else -1
        return 0

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        board = state.board
        return all(board[i] == 0 for i in range(self.pits_per_player)) or \
               all(board[i] == 0 for i in range(self.pits_per_player + 1, len(board) - 1))

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move
    
    def random_move_generator(self):
        """Generate a random valid move for the current player."""
        valid = self.valid_moves()
        if valid:
            return random.choice(valid)
        return None

##### **Minimax and AlphaBeta**

The below are minimax search

In [5]:
def minmax_decision(state, game, maxdepth):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state, depth =1):
        if game.terminal_test(state) or depth == maxdepth:
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), depth + 1))
        return v

    def min_value(state, depth = 1):
        if game.terminal_test(state) or depth == maxdepth:
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), depth + 1))
        return v

    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))


# def alpha_beta_cutoff_search(state, game, d=4, cutoff_test=None, eval_fn=None):
#     """Search game to determine best action; use alpha-beta pruning.
#     This version cuts off search and uses an evaluation function."""

#     player = game.to_move(state)

#     # Functions used by alpha_beta
#     def max_value(state, alpha, beta, depth):
#         if cutoff_test(state, depth):
#             return eval_fn(state)
#         v = -np.inf
#         for a in game.actions(state):
#             v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
#             if v >= beta:
#                 return v
#             alpha = max(alpha, v)
#         return v

#     def min_value(state, alpha, beta, depth):
#         if cutoff_test(state, depth):
#             return eval_fn(state)
#         v = np.inf
#         for a in game.actions(state):
#             v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
#             if v <= alpha:
#                 return v
#             beta = min(beta, v)
#         return v

#### **Random vs. AI**

This code will return the win percentage of 100 games between a player who chooses random moves, and an AI player

In [6]:
games = 100
p1_wins = 0  
p2_wins = 0 
total_moves = 0
num_plies = 5

def random_choice(state):
    return random.choice(game.actions(state))

def minimax_choice(state):
    return minmax_decision(state, game, num_plies)

start_time = tm.time()    

for _ in range(games):
    game = MancalaGame() 
    state = game.initial
    moves = 0  
        
    while not game.terminal_test(state):
        moves += 1
        if state.to_move == 1:
            move = minimax_choice(state)
        else: 
            move = random_choice(state)

        state = game.result(state, move)
        

    total_moves += moves
    
    if state.board[game.pits_per_player] > state.board[-1]:
        p1_wins += 1
    else:
        p2_wins += 1
        
end_time = tm.time()

elapsed_time = end_time - start_time

p1_win_percentage = (p1_wins / games) * 100
p2_win_percentage = (p2_wins / games) * 100
average_moves = total_moves / games

print(f"Player 1 (AI) won: {p1_win_percentage:.2f}% of the time")
print(f"Player 2 (Random) won: {p2_win_percentage:.2f}% of the time")
print(f"On average, it took {average_moves:.2f} moves to win.")
print(f"Elapsed Time {elapsed_time:.2f} sec.")

Player 1 (AI) won: 95.00% of the time
Player 2 (Random) won: 5.00% of the time
On average, it took 41.92 moves to win.
Elapsed Time 23.30 sec.


#### AI Player Analysis

The AI player based on the winrate is much better than random chance. The random chance players would win roughly 50% of the time, whereas the AI Player wins around 95% of the time. In terms of the amount of moves, the difference is not quite as large. However, when compared, the AI vs Random games would take roughly 10 less moves to reach a terminal state. 

#### **Imported Alpha Beta from Games4e**

This is AlphaBeta vs Random at a depth of 5 plies

In [8]:
games = 100
p1_wins = 0  
p2_wins = 0 
total_moves = 0
num_plies = 5

def random_choice(state):
    return random.choice(game.actions(state))

def eval_fn(state):
    return game.utility(state, state.to_move)

def ab_choice(state):
    return alpha_beta_cutoff_search(state, game, d=num_plies, eval_fn=eval_fn)

start_time = tm.time()    

for _ in range(games):
    game = MancalaGame() 
    state = game.initial
    moves = 0  
        
    while not game.terminal_test(state):
        moves += 1
        if state.to_move == 1:
            move = ab_choice(state)
        else: 
            move = random_choice(state)

        state = game.result(state, move)
        

    total_moves += moves
    
    if state.board[game.pits_per_player] > state.board[-1]:
        p1_wins += 1
    else:
        p2_wins += 1
        
end_time = tm.time()

elapsed_time = end_time - start_time

p1_win_percentage = (p1_wins / games) * 100
p2_win_percentage = (p2_wins / games) * 100
average_moves = total_moves / games

print(f"Player 1 (AI) won: {p1_win_percentage:.2f}% of the time")
print(f"Player 2 (Random) won: {p2_win_percentage:.2f}% of the time")
print(f"On average, it took {average_moves:.2f} moves to win.")
print(f"Elapsed Time {elapsed_time:.2f} sec.")

Player 1 (AI) won: 99.00% of the time
Player 2 (Random) won: 1.00% of the time
On average, it took 34.82 moves to win.
Elapsed Time 4.09 sec.


#### Step 7 Analysis

The results are different between the alpha beta player and the minimax player. The alphabeta player ran much faster and had a higher % of wins, although that is likely due to chance. The alphabeta player took 4.09 sec for 100 games, whereas the regular minimax player took 23.30 seconds. This shows how effective alpha beta pruning is at reducing the runtime.

### Extra Credit

In [9]:
games = 100
p1_wins = 0  
p2_wins = 0 
total_moves = 0
num_plies = 10

def random_choice(state):
    return random.choice(game.actions(state))

def eval_fn(state):
    return game.utility(state, state.to_move)

def ab_choice(state):
    return alpha_beta_cutoff_search(state, game, d=num_plies, eval_fn=eval_fn)

start_time = tm.time()    

for _ in range(games):
    game = MancalaGame() 
    state = game.initial
    moves = 0  
        
    while not game.terminal_test(state):
        moves += 1
        if state.to_move == 1:
            move = ab_choice(state)
        else: 
            move = random_choice(state)

        state = game.result(state, move)
        

    total_moves += moves
    
    if state.board[game.pits_per_player] > state.board[-1]:
        p1_wins += 1
    else:
        p2_wins += 1
        
end_time = tm.time()

elapsed_time = end_time - start_time

p1_win_percentage = (p1_wins / games) * 100
p2_win_percentage = (p2_wins / games) * 100
average_moves = total_moves / games

print(f"Player 1 (AI) won: {p1_win_percentage:.2f}% of the time")
print(f"Player 2 (Random) won: {p2_win_percentage:.2f}% of the time")
print(f"On average, it took {average_moves:.2f} moves to win.")
print(f"Elapsed Time {elapsed_time:.2f} sec.")

Player 1 (AI) won: 9.00% of the time
Player 2 (Random) won: 91.00% of the time
On average, it took 47.03 moves to win.
Elapsed Time 168.11 sec.


#### Extra Credit Analysis

A single game takes 1.6811 sec to run to completion. The increasing number of plies hurts the performace for the AI player becasue at a depth of 5 plies the winrate was 99%, where as at a depth of 10 plies it was 9%.